In [ ]:
from anytree import Node, RenderTree
from random import sample
from sklearn import tree

lines = []
tree_cols = []
decision_tree = None
children_left = None
children_right= None
feature = None
threshold = None
fraud_nodes = []
is_leaves = []
rules = {}
    
def make_features(df):
        tree_cols = sample(df.columns,len(df.columns)//5)
        X_train = df[tree_cols].copy().replace(np.nan, -999)
        y_train = df['isFraud'].copy()
        # Create Decision Tree with max_depth = 3
        decision_tree = tree.DecisionTreeClassifier(max_depth = 3, criterion='entropy')
        decision_tree.fit(X_train, y_train)
        lines = tree.export_text(self.decision_tree, feature_names = self.tree_cols).splitlines()
        # The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
        n_nodes = decision_tree.tree_.node_count
        children_left = self.decision_tree.tree_.children_left
        children_right = self.decision_tree.tree_.children_right
        feature = self.decision_tree.tree_.feature
        threshold = self.decision_tree.tree_.threshold
        node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
        is_leaves = np.zeros(shape=n_nodes, dtype=bool)
        stack = [(0, -1)]  # seed is the root node id and its parent depth
        while len(stack) > 0:
            node_id, parent_depth = stack.pop()
            node_depth[node_id] = parent_depth + 1

        # If we have a test node
            if (self.children_left[node_id] != self.children_right[node_id]):
                stack.append((self.children_left[node_id], parent_depth + 1))
                stack.append((self.children_right[node_id], parent_depth + 1))
            else:
                self.is_leaves[node_id] = True  
        self.fraud_nodes = list(filter(lambda x: x[0][1]>x[0][0] ,self.decision_tree.tree_.value)) 
        
    def get_tree(self):
        return self.decision_tree
    
    def get_cols_used(self):
        return self.tree_cols
    
    def get_value(self):
        return self.decision_tree.tree_.value
    
    def get_leaves(self):
        return self.is_leaves
    
    def get_rules(self):
        return self.rules
    
    def build_rules(self):
        leaf_indices = np.where(self.get_leaves())
        leave_id = list(filter(lambda x: self.get_value()[x][0][1]>self.get_value()[x][0][0] ,leaf_indices[0]))
        print(leave_id)
        paths ={}
        
        for leaf in np.unique(leave_id):
            path_leaf = []
            self.find_path(0, path_leaf, leaf)
            paths[leaf] = np.unique(np.sort(path_leaf))
        
        for key in paths:
            self.rules[key] = self.get_rule(paths[key], self.tree_cols)
            if self.rules[key] == '':
                self.rules.pop(key)
            
    def find_path(self, node_numb, path, x):
        path.append(node_numb)
        if node_numb == x:
            return True
        if node_numb!= 0 and self.decision_tree.tree_.impurity[node_numb]<0.5:
            path.remove(node_numb)
            return False
        left = False
        right = False
        if (self.children_left[node_numb] !=-1):
            left = self.find_path(self.children_left[node_numb], path, x)
        if (self.children_right[node_numb] !=-1):
            right = self.find_path(self.children_right[node_numb], path, x)
        if left or right :
            return True
        path.remove(node_numb)
        return False

    def get_rule(self,path, column_names):
        mask = ''
        for index, node in enumerate(path):
        #We check if we are not in the leaf
            if index!=len(path)-1:
            # Do we go under or over the threshold ?
                if (self.children_left[node] == path[index+1]):
                    mask += "({}<= {}) \t ".format(column_names[self.feature[node]], self.threshold[node])
                else:
                    mask += "({}> {}) \t ".format(column_names[self.feature[node]], self.threshold[node])
    # We insert the & at the right places
        mask = mask.replace("\t", "&", mask.count("\t") - 1)
        mask = mask.replace("\t", "")
        return mask  
    
    def make_feature(self,df,it = 1):
        for key in self.rules.keys():
            df['fraudLeaf'+str(key)] = df.eval(self.rules[key])
        fraudLeaves = list(filter(lambda x: 'fraudLeaf' in x,df.columns))
        print(fraudLeaves)
        return df[fraudLeaves].any(axis = 1)     
    